In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split


In [3]:
# load the data from a local CSV file
data = pd.read_csv("C:/Users/user/Documents/GitHub/R5D5-Project/model/ML_Model_Dataset_03/complete_data_model_NBA_Player_Stats_Playoffs.csv")

In [4]:
data.head()

,Rk,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,BLK,TOV,PF,PTS,salary,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG
0,2,28,7,5,16.3,1.3,3.0,0.429,0.0,0.0,...,0.1,0.6,1.7,3.4,0.639241,1,0,0,0,0
1,3,24,18,18,34.1,5.8,9.7,0.594,0.0,0.1,...,0.7,2.1,3.1,14.8,0.077918,1,0,0,0,0
2,4,23,1,0,5.0,2.0,2.0,1.000,0.0,0.0,...,0.0,0.0,0.0,5.0,0.065652,0,0,0,0,1
3,5,26,12,5,25.4,3.1,6.8,0.451,1.6,4.0,...,0.3,0.8,1.8,8.3,0.052227,0,0,0,0,1
4,7,28,12,1,18.4,2.4,4.3,0.569,0.3,1.0,...,0.6,0.8,1.8,6.0,0.218796,0,1,0,0,0


In [5]:
# print the column names
print(data.columns)

Index(['Rk', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'salary', 'Pos_C', 'Pos_PF',
       'Pos_PG', 'Pos_SF', 'Pos_SG'],
      dtype='object')
